In [5]:
# Import the necessary modules
import requests
import arcpy
import zipfile
import imageio
import os

In [2]:
# Set the data URL 
data = 'http://services.nacse.org/prism/data/public/normals/4km/ppt/'

In [3]:
# Use loop to download data and extraxt them to original data folder
for month in range(1,13):
    url = data + str(month)
    print(url)
    r = requests.get(url, allow_redirects=True)
    open(f'{str(month)}.zip', 'wb').write(r.content)
    with zipfile.ZipFile(open(f'C:/Users/user/{str(month)}.zip', 'rb')) as f:
        f.extractall('D:/2021-spring/ArcGIS/Lab2/Cube/original')

http://services.nacse.org/prism/data/public/normals/4km/ppt/1
http://services.nacse.org/prism/data/public/normals/4km/ppt/2
http://services.nacse.org/prism/data/public/normals/4km/ppt/3
http://services.nacse.org/prism/data/public/normals/4km/ppt/4
http://services.nacse.org/prism/data/public/normals/4km/ppt/5
http://services.nacse.org/prism/data/public/normals/4km/ppt/6
http://services.nacse.org/prism/data/public/normals/4km/ppt/7
http://services.nacse.org/prism/data/public/normals/4km/ppt/8
http://services.nacse.org/prism/data/public/normals/4km/ppt/9
http://services.nacse.org/prism/data/public/normals/4km/ppt/10
http://services.nacse.org/prism/data/public/normals/4km/ppt/11
http://services.nacse.org/prism/data/public/normals/4km/ppt/12


In [3]:
# Set the work environment
arcpy.env.workspace = r'D:\2021-spring\ArcGIS\Lab2\Cube'

In [17]:
# Set the parameter for creating mosaic dataset
in_workspace = r'D:\2021-spring\ArcGIS\Lab2\Cube\Cube.gdb'
mosaicdataset = 'lab2cube1'
coordinate_system = arcpy.SpatialReference("c:/coordsystems/NAD 1983.prj")

In [18]:
# Create Mosaic Dataset
arcpy.management.CreateMosaicDataset(in_workspace, mosaicdataset, coordinate_system, None, '', "NONE", None)

<Result 'D:\\2021-spring\\ArcGIS\\Lab2\\Cube\\Cube.gdb\\lab2cube1'>

In [7]:
# Convert .bil data into TIFF
arcpy.env.workspace = r'D:\2021-spring\ArcGIS\Lab2\Cube\original'
outgdb = r'D:\2021-spring\ArcGIS\Lab2\Cube\Cube.gdb'
for bil in arcpy.ListRasters():
    arcpy.conversion.RasterToOtherFormat(bil, outgdb, "TIFF")
    print(bil)

PRISM_ppt_30yr_normal_4kmM2_01_bil.bil
PRISM_ppt_30yr_normal_4kmM2_02_bil.bil
PRISM_ppt_30yr_normal_4kmM2_03_bil.bil
PRISM_ppt_30yr_normal_4kmM2_04_bil.bil
PRISM_ppt_30yr_normal_4kmM2_05_bil.bil
PRISM_ppt_30yr_normal_4kmM2_06_bil.bil
PRISM_ppt_30yr_normal_4kmM2_07_bil.bil
PRISM_ppt_30yr_normal_4kmM2_08_bil.bil
PRISM_ppt_30yr_normal_4kmM2_09_bil.bil
PRISM_ppt_30yr_normal_4kmM2_10_bil.bil
PRISM_ppt_30yr_normal_4kmM2_11_bil.bil
PRISM_ppt_30yr_normal_4kmM2_12_bil.bil


In [19]:
# Add the TIFF files into mosaic dataset
arcpy.env.workspace = r'D:\2021-spring\ArcGIS\Lab2\Cube\Cube.gdb'
for tiff in arcpy.ListRasters():
    arcpy.management.AddRastersToMosaicDataset(mosaicdataset, "Raster Dataset", tiff)
    print(tiff)

PRISM_ppt_30yr_normal_4kmM2_01_bil
PRISM_ppt_30yr_normal_4kmM2_02_bil
PRISM_ppt_30yr_normal_4kmM2_03_bil
PRISM_ppt_30yr_normal_4kmM2_04_bil
PRISM_ppt_30yr_normal_4kmM2_05_bil
PRISM_ppt_30yr_normal_4kmM2_06_bil
PRISM_ppt_30yr_normal_4kmM2_07_bil
PRISM_ppt_30yr_normal_4kmM2_08_bil
PRISM_ppt_30yr_normal_4kmM2_09_bil
PRISM_ppt_30yr_normal_4kmM2_10_bil
PRISM_ppt_30yr_normal_4kmM2_12_bil


In [21]:
# Create a Variable field in the Footprints table of mosaic dataset
arcpy.management.CalculateField(mosaicdataset, "variable", '"prism"', "PYTHON3", '', "TEXT")

<Result 'D:\\2021-spring\\ArcGIS\\Lab2\\Cube\\Cube.gdb\\lab2cube1'>

In [22]:
# Create a Timestamp field in the Footprints table of mosaic dataset
arcpy.management.CalculateField(mosaicdataset, "Time", "DateAdd(Date(1998,0,1), $feature.OBJECTID-1,'year')", "ARCADE", '', "DATE")

<Result 'D:\\2021-spring\\ArcGIS\\Lab2\\Cube\\Cube.gdb\\lab2cube1'>

In [11]:
# Execute the build mutidimensional info tool to format the two fields above correctly 
arcpy.md.BuildMultidimensionalInfo(mosaicdataset, "variable", "Time # #", None)

<Result 'D:\\2021-spring\\ArcGIS\\Lab2\\Cube\\Cube.gdb\\lab2cube'>

In [13]:
# Make Multidimensional Raster Layer 
arcpy.md.MakeMultidimensionalRasterLayer(mosaicdataset, "lab2cube_MultidimLayer", "prism", "ALL", None, None, "", "", "", None, "", "", "DIMENSIONS")

<Result 'lab2cube_MultidimLayer'>

In [14]:
# Create Space Time Cube and save it into disk
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("lab2cube_MultidimLayer", "lab2cube.nc", "ZEROS")

<Result 'D:\\2021-spring\\ArcGIS\\Lab2\\Cube\\lab2cube.nc'>

In [21]:
# Download all the images from ArcPro layouts
aprx = arcpy.mp.ArcGISProject(r'D:\2021-spring\ArcGIS\Lab2\Cube\Cube.aprx')
for layout in range(11,22):
    png = aprx.listLayouts(layout)[0]
    png.exportToGIF(f'D:/2021-spring/ArcGIS/Lab2/Cube/Output/{layout}.gif')

In [29]:
# Get all the dirctories of images
location = r'D:\2021-spring\ArcGIS\Lab2\Cube\Output' 
pics = []
for file in os.listdir(location):
    pics.append(f"{location}\\{file}") 

In [31]:
# Combine all the images and save it as a GIF
gifs = []    
for pic in pics:
    gifs.append(imageio.imread(pic)) 
imageio.mimwrite(r'D:\2021-spring\ArcGIS\Lab2\Cube\Output\GIF.gif', gifs, fps=5)